## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pevek,RU,69.7008,170.3133,37.45,70,55,1.39,broken clouds
1,1,Cape Town,ZA,-33.9258,18.4232,61.21,77,75,4.00,light rain
2,2,Missoula,US,47.0166,-113.8009,66.60,30,88,7.67,overcast clouds
3,3,Airai,TL,-8.9266,125.4092,55.11,89,66,3.91,broken clouds
4,4,Imbituba,BR,-28.2400,-48.6703,60.28,66,0,1.72,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
preferred_min_temp = float(input("Please enter the minimum perferred temperature for your travel destination. "))
preferred_max_temp = float(input("Please enter the maximum perferred temperature for your travel destination. "))

Please enter the minimum perferred temperature for your travel destination. 75
Please enter the maximum perferred temperature for your travel destination. 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= preferred_min_temp) & 
                                       (city_data_df["Max Temp"] < preferred_max_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Gigmoto,PH,13.7796,124.3900,82.44,85,100,7.76,overcast clouds
11,11,Bethel,US,41.3712,-73.4140,79.20,49,20,18.41,few clouds
13,13,Pataskala,US,39.9956,-82.6743,79.02,57,1,1.01,clear sky
15,15,Hithadhoo,MV,-0.6000,73.0833,81.90,78,100,17.83,overcast clouds
18,18,Hilo,US,19.7297,-155.0900,87.28,76,58,5.01,broken clouds
20,20,Mamallapuram,IN,12.6264,80.1722,86.05,57,80,10.63,broken clouds
21,21,Kapaa,US,22.0752,-159.3190,82.38,72,97,1.99,overcast clouds
22,22,Gushikawa,JP,26.3544,127.8686,80.60,90,75,12.97,broken clouds
31,31,Dong Hoi,VN,17.4833,106.6000,78.82,88,86,3.22,overcast clouds
32,32,Atuona,PF,-9.8000,-139.0333,79.07,72,12,14.45,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             215
City                   215
Country                215
Lat                    215
Lng                    215
Max Temp               215
Humidity               215
Cloudiness             215
Wind Speed             215
Current Description    215
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Gigmoto,PH,82.44,overcast clouds,13.7796,124.3900,
11,Bethel,US,79.20,few clouds,41.3712,-73.4140,
13,Pataskala,US,79.02,clear sky,39.9956,-82.6743,
15,Hithadhoo,MV,81.90,overcast clouds,-0.6000,73.0833,
18,Hilo,US,87.28,broken clouds,19.7297,-155.0900,
20,Mamallapuram,IN,86.05,broken clouds,12.6264,80.1722,
21,Kapaa,US,82.38,overcast clouds,22.0752,-159.3190,
22,Gushikawa,JP,80.60,broken clouds,26.3544,127.8686,
31,Dong Hoi,VN,78.82,overcast clouds,17.4833,106.6000,
32,Atuona,PF,79.07,few clouds,-9.8000,-139.0333,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    try:
        hotel_response = requests.get(base_url, params).json()
        
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found.....Skipping")         

Hotel not found.....Skipping
Hotel not found.....Skipping
Hotel not found.....Skipping
Hotel not found.....Skipping
Hotel not found.....Skipping
Hotel not found.....Skipping
Hotel not found.....Skipping
Hotel not found.....Skipping
Hotel not found.....Skipping
Hotel not found.....Skipping
Hotel not found.....Skipping
Hotel not found.....Skipping
Hotel not found.....Skipping
Hotel not found.....Skipping
Hotel not found.....Skipping


In [25]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]

In [28]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Gigmoto,PH,82.44,overcast clouds,13.7796,124.3900,Cj's Travellers Inn
11,Bethel,US,79.20,few clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
13,Pataskala,US,79.02,clear sky,39.9956,-82.6743,SD
15,Hithadhoo,MV,81.90,overcast clouds,-0.6000,73.0833,Scoop Guest House
18,Hilo,US,87.28,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
20,Mamallapuram,IN,86.05,broken clouds,12.6264,80.1722,Radisson Blu Resort Temple Bay Mamallapuram
21,Kapaa,US,82.38,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
22,Gushikawa,JP,80.60,broken clouds,26.3544,127.8686,Kirari Guest House
31,Dong Hoi,VN,78.82,overcast clouds,17.4833,106.6000,Nam Long Hotel
32,Atuona,PF,79.07,few clouds,-9.8000,-139.0333,Villa Enata


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [39]:
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))